In [1]:
import sys
sys.path.append('../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
from systematics_module import *
import os
from numpy.lib.recfunctions import append_fields
import scipy.stats

import matplotlib.pyplot as plt
import numpy as np
from run_systematics import sys_iteration, weightmultiply, fitting_allSP, calling_sysMap
from sys_functions import *

In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [3]:
import numpy as np

path = '/fs/scratch/PCON0008/warner785/bwarner/'

cmass_ngc = esutil.io.read(path+'galaxy_DR12v5_CMASS_North.fits.gz')
cmass_sgc = esutil.io.read(path+'galaxy_DR12v5_CMASS_South.fits.gz')
cmass_train = keepGoodRegion(cmass_sgc)

cmass = np.hstack([cmass_ngc, cmass_sgc])

print(np.sum(cmass_train['WEIGHT_CP']))

58477.0


In [6]:
# calling map 
path = '/fs/scratch/PCON0008/warner785/bwarner/mocks/'
LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAO_SPT_ring.fits') #BAO is different measurement, may be different from this analysis
# 'Y1LSSmask_v2_redlimcut_il22_seeil4.0_4096ring.fits'
#GoldMask = callingEliGoldMask()
LSSMask = LSSGoldmask
GoldMask = LSSGoldmask

pixra, pixdec = hp.pix2ang(nside=4096,ipix=GoldMask['PIXEL'],nest=True,lonlat=True)
LSSMask = LSSMask[pixdec >-3.0 ]
GoldMask_spt = GoldMask[ pixdec < -3.0 ]
pixarea = hp.nside2pixarea( 4096, degrees = True)
sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
SPTMaparea = pixarea * sptnpix

sys_path = '/fs/scratch/PCON0008/warner785/bwarner/SPmaps_cut/'
sysMap = fitsio.read(sys_path+'FWHM_FLUXRADr.fits')
sys_mask = (sysMap['SIGNAL'] < 4.5)
map_mask = np.in1d( GoldMask_spt['PIXEL'],  sysMap['PIXEL'][sys_mask])
GoldMask_spt_sysmasked = GoldMask_spt[map_mask]
sptnpix_sys = np.sum(GoldMask_spt_sysmasked['FRACGOOD']) 

SPTMaparea_sys = pixarea * sptnpix_sys
print(SPTMaparea_sys, SPTMaparea)

3221.208600509108 3254.3984021391675


In [14]:
# <Survey Area> 
#-This is a rough estimation
#training region (dec > -10.0):  296 sq.deg

#rootdir = '../output/sfd_train_photo_SamPle_nozband_ran10_3_full_tol1e5/'
dmass_spt = calling_catalog('/fs/scratch/PCON0008/warner785/bwarner/dmass_spt.fits')
dmass_spt = dmass_spt[ dmass_spt['CMASS_PROB'] > 0.01 ]
#dmass_st82 = fitsio.read('/fs/scratch/PCON0008/warner785/bwarner/matched_cmass.fits') # cmass data
dmass_st82 = fitsio.read('../output/test/train_cat/y3/dmass_st82_part1.fits') # dmass data
dmass_st82 =dmass_st82[ dmass_st82['CMASS_PROB'] > 0.01]

# des in training region:
train = esutil.io.read('/fs/scratch/PCON0008/warner785/bwarner/des_training.fits')
train = keepGoodRegion(train)

dmass_spt = keepGoodRegion(dmass_spt)
dmass_st82 = keepGoodRegion(dmass_st82)

#print(dmass_st82.dtype.names)

print("dmass_st82 sum: ",np.sum(dmass_st82['CMASS_PROB']))
print("dmass_spt sum: ",np.sum(dmass_spt['CMASS_PROB']))
print("des training sum: ",train.size)

import pandas as pd
desData = pd.DataFrame(dmass_spt)
goldData = pd.DataFrame(GoldMask_spt)
matched = pd.merge(desData, goldData, left_on='HPIX_4096', right_on='PIXEL', how='left')
matched_arr = matched.to_records(index=False)

from utils import mergeCatalogsUsingPandas
dmass_spt_hpix = mergeCatalogsUsingPandas(des=dmass_spt, gold=GoldMask_spt, how = 'right', 
                                      left_key='HPIX', right_key = 'PIXEL', suffixes = ['','_GOLD'], 
                                      left_index=False, right_index = False)

#BOSSCleanMaskSGC = callingBOSSmask(option='SGC')
#BOSSCleanMaskNGC = callingBOSSmask(option='NGC')
#BOSSSGCMaparea = BOSSCleanMaskSGC.size * pixarea
#BOSSNGCMaparea = BOSSCleanMaskNGC.size * pixarea
#print BOSSSGCMaparea, BOSSNGCMaparea

Resulting catalog size
DMASS= 341117.6414259936
dmass_st82 sum:  20206.04487947579
dmass_spt sum:  341117.6414259936
des training sum:  2740075


ValueError: Big-endian buffer not supported on little-endian compiler

In [ ]:
# simple method first --- possion errors

In [18]:
# splitting CMASS in training region-sized samples:

path = '/fs/scratch/PCON0008/warner785/bwarner/'
cmass_sgc = esutil.io.read(path+'galaxy_DR12v5_CMASS_South.fits.gz')
train = esutil.io.read('/fs/scratch/PCON0008/warner785/bwarner/des_training.fits')
train = keepGoodRegion(train)

print(train['HPIX_4096'].size)
print(cmass_sgc.dtype.names)

2740075
('RA', 'DEC', 'RUN', 'RERUN', 'CAMCOL', 'FIELD', 'ID', 'ICHUNK', 'IPOLY', 'ISECT', 'FRACPSF', 'EXPFLUX', 'DEVFLUX', 'PSFFLUX', 'MODELFLUX', 'FIBER2FLUX', 'R_DEV', 'EXTINCTION', 'PSF_FWHM', 'AIRMASS', 'SKYFLUX', 'EB_MINUS_V', 'IMAGE_DEPTH', 'IMATCH', 'Z', 'WEIGHT_FKP', 'WEIGHT_CP', 'WEIGHT_NOZ', 'WEIGHT_STAR', 'WEIGHT_SEEING', 'WEIGHT_SYSTOT', 'NZ', 'COMP', 'PLATE', 'FIBERID', 'MJD', 'FINALN', 'TILE', 'SPECTILE', 'ICOLLIDED', 'INGROUP', 'MULTGROUP')


In [24]:
NSIDE = 512
NPIX = hp.nside2npix(NSIDE)
print(NPIX)

# pixel area function -- lookup
# https://healpy.readthedocs.io/en/latest/generated/healpy.pixelfunc.nside2pixarea.html#healpy.pixelfunc.nside2pixarea

3145728


In [ ]:
def number_gal(sysMap, dmass_chron, dmass_chron_weights, sys_weights = False, mocks = False, iterative = False): # apply systematic weights here
    
    minimum = np.percentile(sysMap['SIGNAL'][sysMap['SIGNAL']!=hp.UNSEEN], 1)
    maximum = np.percentile(sysMap['SIGNAL'][sysMap['SIGNAL']!=hp.UNSEEN], 99)

    pbin, pstep = np.linspace( minimum, maximum, 13, retstep=True)
    pcenter = pbin[:-1] + pstep/2

    x = np.full(hp.nside2npix(4096), hp.UNSEEN)
    x[sysMap['PIXEL']] = sysMap['SIGNAL']
    #print("sysmap: ", sysMap['SIGNAL'])
    if mocks != True:
        sysval_gal = x[dmass_chron['HPIX_4096']].copy()
        
    else: 
        sysval_gal = x[dmass_chron].copy()
    
    if mocks != True:
        if sys_weights == True:
            print("weights being applied...")
            h,_ = np.histogram(sysval_gal[sysval_gal != hp.UNSEEN], bins=pbin, weights = dmass_chron["CMASS_PROB"][sysval_gal != hp.UNSEEN]*dmass_chron_weights[sysval_gal != hp.UNSEEN])
        else:
            if iterative != True:
                h,_ = np.histogram(sysval_gal[sysval_gal != hp.UNSEEN], bins=pbin, weights = dmass_chron["CMASS_PROB"][sysval_gal != hp.UNSEEN]) # -- density of dmass sample, not gold sample
        if iterative == True:
            print("weights being applied...")
            h,_ = np.histogram(sysval_gal[sysval_gal != hp.UNSEEN], bins=pbin, weights = dmass_chron["CMASS_PROB"][sysval_gal != hp.UNSEEN]*dmass_chron_weights[sysval_gal != hp.UNSEEN])
    else:
        h,_ = np.histogram(sysval_gal[sysval_gal != hp.UNSEEN], bins=pbin, weights = dmass_chron_weights[sysval_gal != hp.UNSEEN])
    
    return h, sysval_gal

def area_pixels(sysMap, frac_weight, custom):
    
    minimum = np.percentile(sysMap['SIGNAL'][sysMap['SIGNAL']!=hp.UNSEEN], 1)
    maximum = np.percentile(sysMap['SIGNAL'][sysMap['SIGNAL']!=hp.UNSEEN], 99)

    pbin, pstep = np.linspace( minimum, maximum, 13, retstep=True)
    pcenter = pbin[:-1] + pstep/2
    
# number of galaxies in each pixel:
    #if debug != True:
    if custom == True:
        sys_signal = sysMap['SIGNAL']
        area,_ = np.histogram(sys_signal[sys_signal != hp.UNSEEN] , bins=pbin , weights = frac_weight)
    else:   
        sys_signal = sysMap['SIGNAL']
        area,_ = np.histogram(sys_signal[sys_signal != hp.UNSEEN] , bins=pbin , weights = sysMap['FRACDET'][sys_signal != hp.UNSEEN])
            
    #else:  (for SP if other method isn't working)
# number of galaxies in each pixel:
        #debug this to be like the PCA set
        #sys_signal = sysMap['SIGNAL']

        #sys = sysMap
        #mask = np.full(hp.nside2npix(4096), hp.UNSEEN)
        #frac_mask = np.in1d(fracDet["PIXEL"], sys["PIXEL"], assume_unique=False, invert=False)

        #mask[sys["PIXEL"]] = sys["SIGNAL"]
        #frac_sys = mask[fracDet["PIXEL"][frac_mask]]
        #area,_ = np.histogram(frac_sys[frac_sys != hp.UNSEEN] , bins=pbin , weights = fracDet["SIGNAL"][frac_mask][frac_sys != hp.UNSEEN])

    
    return area

def number_density(sysMap, h, area):
    
    minimum = np.percentile(sysMap['SIGNAL'][sysMap['SIGNAL']!=hp.UNSEEN], 1)
    maximum = np.percentile(sysMap['SIGNAL'][sysMap['SIGNAL']!=hp.UNSEEN], 99)

    pbin, pstep = np.linspace( minimum, maximum, 13, retstep=True)
    pcenter = pbin[:-1] + pstep/2

    number_density = []
    for x in range(len(h)):
        den = h[x]/area[x]
        number_density.append(den)

    total_area = 0
    #Normalize based on total number density of used footprint:
    for x in range(len(area)):
        total_area += area[x]

    # total galaxies:
    total_h = 0
    for x in range(len(h)):
        total_h += h[x]

    #normalization: 
    total_num_density = total_h/total_area
    
    # apply normalization: 
    #print(number_density)
    norm_number_density = number_density/total_num_density
    #print(norm_number_density_ran)

    fracerr = np.sqrt(h) #1 / sqrt(number of randoms cmass galaxies in each bin)
    fracerr_norm = (fracerr/area)/total_num_density
    
    return pcenter, norm_number_density, fracerr_norm

In [ ]:
fig,(ax, ax2, ax3) = plt.subplots(1, 3, figsize = (17, 5))

ax.axhspan( mean_n_density_ngc - std_n_density_ngc, mean_n_density_ngc + std_n_density_ngc, alpha=0.3, color='grey')#, label = r'\rm 68% confidence}')
ax.axhspan( mean_n_density_ngc - 2*std_n_density_ngc, mean_n_density_ngc + 2*std_n_density_ngc, alpha=0.1, color='grey')#, label = r'\rm 68 % confidence}')
#ax.axhspan( 1. - 2*defined_sigma_st82/mean, 1 + 2*defined_sigma_st82/mean, alpha=0.1, color='grey', linestyle = None, label = '95 % confidence')
#ax.errorbar( np.arange(n_density_ngc_norm.size), n_density_ngc_norm, yerr = err_ngc, fmt='o' )
ax.plot( np.arange(avg_n_density_ngc.size), avg_n_density_ngc, marker = '.' , linestyle = 'none', color = 'black' )
ax.plot( avg_n_density_ngc.size/2., ntotal_st82/n_density_ngc.mean(), marker = '*', color = 'red',  markersize = 10, linestyle = 'none', label = r'{\rm train}')
ax.plot( avg_n_density_ngc.size/2., ntotal_DMASS/n_density_ngc.mean(), marker = '*', color = 'blue',  markersize = 10, linestyle = 'none', label = r'{\rm DMASS}')

ax.axhline(y = mean_n_density_ngc, linestyle='--', color = 'grey')
ax.set_title(r'{\rm CMASS NGC}', fontsize = 20)
ax.set_ylim(0.85, 1.15)
ax.legend(loc=4, fontsize = 15)
ax.set_ylabel(r'$n_i / \langle n \rangle$', fontsize = 20)
ax.set_xlabel(r'{\rm patch index $i$}', fontsize = 20)
ax.text(0, 1.12, r'{\rm  Patch area }'+ r'${:0.0f} \deg^2$'.format(area_ngc), fontsize = 15)
#ax.text(0, 1.10, r'{\rm  St82 area }'+ r'${:0.0f} \deg^2$'.format(123), fontsize = 15)
ax.tick_params(labelsize = 15)